In [20]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from decimal import Decimal
import datetime

In [1]:
import logging

# tato cast musi ist do gui, kde sa vytvori log subor, a inak to bude pouzivat iba default config
logging.basicConfig(level=logging.INFO,format='%(levelname)s (%(name)s): %(asctime)s >>> %(message)s', datefmt='%d-%b-%y %H:%M:%S', filename='Ilaps_log.txt')


In [2]:
logging.info('This will get logged to a file')

In [3]:
def get_timestamp(strTime):
    """format string time from iolite to timestamp"""
    return datetime.datetime.strptime(strTime, '%Y-%m-%d %H:%M:%S.%f')


def get_difference(start, now):
    """return time in seconds between 2 timestamps"""
    diff = now - start
    return diff.total_seconds()


def get_index(data, time):
    """return closest index of MS time given time in seconds"""
    for i in range(len(data.index)-1):
        if (data.index[i] <= time) and (data.index[i+1]>time):
                return i+1

def get_diff_lst(iolite):
    """return list of times in seconds from start to every start and end of laser ablation for spots"""
    lst = []
    for i in range(1,len(iolite['Timestamp'])-1):
        if (i-4)%5==0:
            lst.append(get_difference(get_timestamp(iolite.loc[i-2,'Timestamp']),get_timestamp(iolite.loc[i,'Timestamp'])))
            lst.append(get_difference(get_timestamp(iolite.loc[i,'Timestamp']),get_timestamp(iolite.loc[i+1,'Timestamp'])))
    lst.append(get_difference(get_timestamp(iolite.loc[i,'Timestamp']),get_timestamp(iolite.loc[i+1,'Timestamp'])))
    return lst

def get_diff_lst_line(iolite):
    """return list of times in seconds from start to every start and end of laser ablation for lines"""
    lst = []
    for i in range(1,len(iolite['Timestamp'])-1):
        if (i-6)%7==0:
            lst.append(get_difference(get_timestamp(iolite.loc[i-2,'Timestamp']),get_timestamp(iolite.loc[i,'Timestamp'])))
            lst.append(get_difference(get_timestamp(iolite.loc[i,'Timestamp']),get_timestamp(iolite.loc[i+1,'Timestamp'])))
    lst.append(get_difference(get_timestamp(iolite.loc[i-2,'Timestamp']),get_timestamp(iolite.loc[i,'Timestamp'])))
    return lst



In [4]:
class DataReader():  
    """
    Reads data into pandas Dataframe from a file.

    Parameters
    ----------
    filetype : str
        Type of the file to read. If not specified, csv is used. 
        Possible options are csv, xlsx and asc.
    instrument : str
        Type of the instrument used for measurement. If not specified, raw data is expected. 
        Possible options are Agilent, and Element.

    Returns
    -------
    data : dataframe
        `data` as a dataframe, which can be passed to MSData.

    """
    def __init__(self, filename, filetype=None, instrument=None):
        
        self.filename = filename
        self.filetype = filetype
        self.instrument = instrument
        self.data = self.read(filename, filetype, instrument)
        
    def __call__(self):
        return self.data
        
    def read(self, filename, filetype, instrument):
        if instrument == 'Element':
            skipfooter = 4
            header = 1
            drop = 9
        elif instrument == 'Agilent':
            skipfooter = 4
            header = 3
            drop = 3
        else:
            skipfooter = 0
            header = 0
            drop = 0

        if filetype == 'xlsx':
            imported = pd.ExcelFile(filename)
            data = imported.parse(0, index_col=0, skipfooter=skipfooter, header=header)
            data = sdata.drop(data.index[:drop], axis=0)

        elif filetype == 'csv':
            data = pd.read_csv(filename, sep=',', index_col=0, skipfooter=skipfooter,
                                    header=header, engine='python')

        elif filetype == 'asc':
            data = pd.read_csv(filename, sep='\t', index_col=0, skipfooter=skipfooter,
                                    header=header, engine='python')
            data = data.drop(data.index[:drop], axis=0)
            data.dropna(axis=1, how='all', inplace=True)
            data = data.apply(pd.to_numeric, errors='coerce')

        else:
            warnings.warn('File type not supported.')

        return data
    

In [5]:
def plot_data(data, isotopes=None, ax=None, *args, **kwargs):
    if ax is None:
        fig, ax = plt.subplots()
    if isotopes is not None:
        ax.plot(data[isotopes], *args, **kwargs)
    else:
        ax.plot(data, *args, **kwargs)
    

In [6]:
class Session():
    pass

In [23]:
class MSData():
    def __init__(self, datareader):
        self.datareader = datareader
        logging.info(f'Reading data {self.datareader.filename}.')
        self.data = self.datareader()
        self.time = np.array(self.data.index)
        self.matrix = self.data.values
        self.isotope_names = np.array(self.data.columns)
        self.isotopes = {}
        
    def __call__(self, isotopes=None, *args, **kwargs):
        plot_data(self.data, isotopes=isotopes, *args, **kwargs)
        
    def __repr__(self):
        res = f'{self.__class__.__name__}\n Analysis time: {self.time.max()}s\n File path: {self.datareader.filename}\n Measured isotopes: {self.isotope_names}'
        return res
    
    def time_to_number(self, time):
        """
        takes time in seconds returns number of measured values
        depends on integration time of MS method
        """
        val = len(self.time[0:np.abs((self.time)-np.abs(time)).argmin()])
        #val = len(d.time[:np.abs((d.time)-np.abs(10)).argmin()])
        if time < 0: val = -val
        return val
    
    def create_isotopes(self):
        for key in self.isotope_names:
            self.isotopes[key] = Isotope(key, self)
    
    def create_selector(self):
        pass
    

In [8]:
class Isotope():   
    """
    Extract specific Isotope data from MSData object.

    Parameters
    ----------
    isotope_name : str
        Isotope to extract from MSData (i.e. Li7)
    ms_data : MSData
        MS data to extract from.
        
    Returns
    -------
    data : array
        `data` interpreted as an array.

    """
    def __init__(self, isotope_name, ms_data, *args, **kwargs):
        self.isotope_name = isotope_name
        self.isotope_number = int(''.join([c for c in self.isotope_name if c.isnumeric()]))
        self.data = ms_data.data[isotope_name].values
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        if isinstance(idx, (int,slice)): return self.data[idx]
        return [self.data[i] for i in idx]
    def __call__(self):
        plt.plot(self.data)
    def __repr__(self):
        res = f'{self.__class__.__name__} {self.isotope_name} ({len(self)} values)\n{self.data[:10]}'
        if len(self)>10: res = res[:-1]+ '...]'
        return res
    def __iter__(self): return iter(self.data)
    def __setitem__(self, i, o): self.data[i] = o
    def __delitem__(self, i): del(self.data[i])
    
    def element(self):
        return ''.join([c for c in self.isotope_name if c.isalpha()])
    
    def peakiefy():
        pass
    
    
        

In [9]:
class Peak():
    def __init__(self, isotope, bounds, ms_data=None, *args, **kwargs):
        if isotope.data is None:
            raise ValueError('Missing data for isotope: {}'.format(isotope.isotope_name))
        self.data = isotope.data[bounds[0]:bounds[1]]
    def __len__(self):
        return len(self.data)
    def __call__(self):
        plt.plot(self.data)
    def __repr__(self):
        pass 

In [10]:
class Background():
    def __init__(self, isotope, laser_on, laser_off, ms_data=None, *args, **kwargs):
        assert isotope.data is not None
        self.bcg_beginning = isotope.data[laser_off[0][0]:laser_off[0][1]]
        self.bcg_end = isotope.data[laser_off[-1][0]:laser_off[-1][1]]
        self.bcg_all = [isotope.data[laser_off[i][0]:laser_off[i][1]] for i in range(len(laser_off))]
        self.bcg_means = [x.mean() for x in self.bcg_all]
    def __call__(self):
        plt.plot(self.bcg_means)
    def __repr__(self):
        return f'{self.__class__.__name__}: {self.bcg_means} '

In [11]:
class ElementalMap():
    def __init__(self, isotope, laser_on, laser_off, ms_data, *args, **kwargs):
        pass
    def __len__(self):
        pass
    def __call__(self):
        plt.plot(self.bcg_means)
    def __repr__(self):
        pass 

In [12]:
class Iolite():
    def __init__(self, path):
        self.data = self.read_iolite(path)
        self.peak_names = self.names_from_iolite(self.data)
        
    def read_iolite(self, path):
        iolite = pd.read_csv(path, sep=",", engine='python')
        return iolite
        
    def names_from_iolite(self, iolite):
        names = list(iolite[' Comment'].dropna())
        return names
    
    
    

In [13]:
class Selector():
    def __init__(self, ms_data, method = 'treshold', s=60, sdmul=10, iolite=None):
        self.possible_methods = ('treshold','iolite')
        self.ms_data = ms_data
        self.filter_line = self.ms_data.data.sum(1)
        self.method = method
        self.iolite = iolite
        self.start = s
        self.sdmul = sdmul
        
    def __call__(self):
        if method not in self.possible_methods:
            warnings.warn(f'{method} not implemented.\n Select from: {self.possible_methods}')
            return  
        elif self.method == 'treshold':
            return self.create_selector_treshold()
        elif self.method == 'iolite':
            return self.create_selector_iolite()
        
    
    def create_selector_iolite(self):
        """select starts and ends of ablation using iolite file """ 
        logging.info('Selecting peak bounds by iolite.')
        
        lst = [x for x in self.iolite.data.loc[:6, ' Comment'] if isinstance(x, str)]
        
        if len(lst) == 2:
            logging.info('Selecting spots.')
            difflst = get_diff_lst(self.iolite.data)
        elif len(lst) == 1:
            logging.info('Selecting lines.')
            difflst = get_diff_lst_line(self.iolite.data)
        else:
            logging.error('Iolite selection failed.')
            
        timeindex = []
        for i in range(0, len(difflst)+1):
            timeindex.append(sum(difflst[:i])+self.start)
        index =[get_index(self.ms_data.data, x) for x in timeindex]
        
        starts = [index[i] for i in range(len(index)) if i %2==0]
        ends = [index[i] for i in range(len(index)) if i %2!=0]
        
        return starts, ends
    
    def create_selector_treshold(self):
        """
        select starts and ends of ablation based on selected element or sum of all using treshold
        calculated from background
        """
        logging.info('Selecting peak bounds by setting treshold.')
        bcg_nr = self.ms_data.time_to_number(self.start)
        bcg = self.filter_line[0:bcg_nr].mean()
        std = self.filter_line[0:bcg_nr].std()
        ind = [True if value > bcg+self.sdmul*std else False for value in self.filter_line]
        ind2 = ind[1:]; ind2.append(False)
        index = [i for i in range(0,len(ind)) if ind[i]!=ind2[i]]
        
        starts = [index[i] for i in range(len(index)) if i %2==0]
        ends = [index[i] for i in range(len(index)) if i %2!=0]
        
        return starts, ends
        



In [14]:
class Quantifier():
    pass

In [24]:
reader = DataReader(filename='/Users/nikadilli/code/Ilaps/test_data/mapa1c_data.csv', filetype='csv', instrument='raw')

In [25]:
d = MSData(reader)


In [338]:
io = Iolite('/Users/nikadilli/code/Ilaps/test_data/mapa1.Iolite.csv')
io.data

,Timestamp,Sequence Number,SubPoint Number,Vertix Number,Comment,X,Y,Intended X,Intended Y,Scan Velocity,Laser State,Laser Rep. Rate,Spot Type,Spot Size,Spot Angle,MFC1,MFC2,Cell Pressure,Fluence At Sample
0,2018-12-17 15:00:37.681,1.0,1.0,NaN,Line 1,51600,28600,NaN,NaN,NaN,Off,0,Manual: G,40,0,0,0,0,NaN
1,2018-12-17 15:00:37.681,NaN,NaN,1.0,NaN,51600,28600,NaN,NaN,40.0,Off,0,Manual: G,40,0,0,0,0,NaN
2,2018-12-17 15:00:37.681,NaN,NaN,NaN,NaN,51600,28600,51600.0,28600.0,NaN,Off,0,Manual: G,40,0,0,0,0,NaN
3,2018-12-17 15:00:37.681,NaN,NaN,NaN,NaN,51600,28600,NaN,NaN,NaN,On,10,Manual: G,40,0,0,0,0,NaN
4,2018-12-17 15:00:37.681,NaN,NaN,2.0,NaN,51600,28600,NaN,NaN,40.0,On,10,Manual: G,40,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,2018-12-17 16:17:50.193,NaN,NaN,NaN,NaN,51600,30960,51600.0,30960.0,NaN,Off,0,Manual: G,40,0,0,0,0,NaN
416,2018-12-17 16:17:50.193,NaN,NaN,NaN,NaN,51600,30960,NaN,NaN,NaN,On,10,Manual: G,40,0,0,0,0,NaN
417,2018-12-17 16:17:50.193,NaN,NaN,2.0,NaN,51600,30960,NaN,NaN,40.0,On,10,Manual: G,40,0,0,0,0,NaN
418,2018-12-17 16:18:55.332,NaN,NaN,NaN,NaN,54200,30960,54200.0,30960.0,NaN,On,10,Manual: G,40,0,0,0,0,NaN


In [27]:
select = Selector(d, 'treshold', 5, 5)

In [28]:
starts, ends = select.create_selector_treshold()

In [344]:
d.create_isotopes()

In [345]:
d.isotopes

{'Na23': Isotope Na23 (5870 values)
 [52089.86 51186.78 44666.09 47374.34 46571.85 47374.34 49581.42 45970.
  48678.49 45167.58...], 'Mg24': Isotope Mg24 (5870 values)
 [100.   0.   0.   0.   0.   0.   0. 100.   0. 100....], 'Al27': Isotope Al27 (5870 values)
 [  0.     0.   100.   200.   300.     0.     0.     0.     0.   500.01...], 'Si28': Isotope Si28 (5870 values)
 [44465.73 41156.31 45268.12 45268.12 41256.59 38348.9  41757.97 40253.87
  44265.13 42560.22...], 'P31': Isotope P31 (5870 values)
 [360.   340.   380.   340.   400.01 300.   200.   300.   320.   360.  ...], 'Ca43': Isotope Ca43 (5870 values)
 [ 44.44  22.22  88.89   0.    66.67  88.89  88.89   0.   133.33  44.44...], 'Ti47': Isotope Ti47 (5870 values)
 [ 0.  0.  0.  0.  0.  0.  0.  0. 20.  0....], 'Cr52': Isotope Cr52 (5870 values)
 [100.     0.   700.02   0.     0.   200.   100.   300.   200.   100.  ...], 'Mn55': Isotope Mn55 (5870 values)
 [100.   0. 100.   0. 100.   0.   0.   0. 100. 100....], 'Fe56': Isotope Fe56 

In [319]:
d.data['Na23'].values

array([52089.86, 51186.78, 44666.09, ..., 57911.05, 55702.75, 58714.14])